.

Data_Engineering_TIL(20201009)

[실습시 참고한 자료]

1) 'Install RabbitMq on Amamzon EC2 (Amazon Linux 2)' 블로그 글

** URL : https://dev.to/ashutosh049/install-rabbitmq-on-amamzon-ec2-amazon-linux-2-3dpd

2) '[Rabbitmq] 1. 초보를 위한 RabbitMQ 후딱 설치하고 설정하기' 블로그 글

** URL : http://abh0518.net/tok/?p=384

[실습내용]

- 실습환경은 Amazon Linux 2에서 진행한다.


- RabbitMq를 설치하기 전에 패키지 디펜턴시를 맞춰줘야 한다.

그래서 RabbitMq를 설치하기 전에 아래 3개의 패키지들이 사용가능하도록 할 것이다.

1) erlang

2) socat

3) logrotate

- EPEL repository를 사용할 수 있도록 설치

일반적인 레포에서는 CentOS, Red Hat Enterprise Linux (RHEL), or Amazon Linux-based distributions 와 같은 운영체제에 설치될 수 있는 모든 패키지들을 제공하지 않기 때문에 EPEL repository를 사용하고자 한다.

그래서 먼저 아래와 같이 EPEL release package for EL 7을 설치하여 EPEL repository를 사용할 수 있도록 해준다.

In [ ]:
[ec2-user@ip-10-1-10-188 ~]$ sudo yum update -y
[ec2-user@ip-10-1-10-188 ~]$ sudo yum install -y https://dl.fedoraproject.org/pub/epel/epel-release-latest-7.noarch.rpm
Loaded plugins: extras_suggestions, langpacks, priorities, update-motd
Existing lock /var/run/yum.pid: another copy is running as pid 11673.
Another app is currently holding the yum lock; waiting for it to exit...
  The other application is: yum
    Memory : 119 M RSS (336 MB VSZ)
    Started: Fri Oct  9 06:10:16 2020 - 00:03 ago
    State  : Running, pid: 11673
epel-release-latest-7.noarch.rpm                                                             |  15 kB  00:00:00
Examining /var/tmp/yum-root-XGv900/epel-release-latest-7.noarch.rpm: epel-release-7-12.noarch
Marking /var/tmp/yum-root-XGv900/epel-release-latest-7.noarch.rpm to be installed
Resolving Dependencies
--> Running transaction check
---> Package epel-release.noarch 0:7-12 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

====================================================================================================================
 Package                   Arch                Version             Repository                                  Size
====================================================================================================================
Installing:
 epel-release              noarch              7-12                /epel-release-latest-7.noarch               24 k

Transaction Summary
====================================================================================================================
Install  1 Package

Total size: 24 k
Installed size: 24 k
Downloading packages:
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : epel-release-7-12.noarch                                                                         1/1
  Verifying  : epel-release-7-12.noarch                                                                         1/1

Installed:
  epel-release.noarch 0:7-12

Complete!

[ec2-user@ip-10-1-10-188 ~]$ sudo yum-config-manager --enable epel
Loaded plugins: extras_suggestions, langpacks, priorities, update-motd
==================================================== repo: epel ====================================================
[epel]
async = True
bandwidth = 0
base_persistdir = /var/lib/yum/repos/x86_64/2
baseurl =
cache = 0
cachedir = /var/cache/yum/x86_64/2/epel
check_config_file_age = True
compare_providers_priority = 80
cost = 1000
deltarpm_metadata_percentage = 100
deltarpm_percentage =
enabled = True
enablegroups = True
exclude =
failovermethod = priority
ftp_disable_epsv = False
gpgcadir = /var/lib/yum/repos/x86_64/2/epel/gpgcadir
gpgcakey =
gpgcheck = True
gpgdir = /var/lib/yum/repos/x86_64/2/epel/gpgdir
gpgkey = file:///etc/pki/rpm-gpg/RPM-GPG-KEY-EPEL-7
hdrdir = /var/cache/yum/x86_64/2/epel/headers
http_caching = all
includepkgs =
ip_resolve =
keepalive = True
keepcache = False
mddownloadpolicy = sqlite
mdpolicy = group:small
mediaid =
metadata_expire = 21600
metadata_expire_filter = read-only:present
metalink = https://mirrors.fedoraproject.org/metalink?repo=epel-7&arch=x86_64&infra=$infra&content=$contentdir
minrate = 0
mirrorlist =
mirrorlist_expire = 86400
name = Extra Packages for Enterprise Linux 7 - x86_64
old_base_cache_dir =
password =
persistdir = /var/lib/yum/repos/x86_64/2/epel
pkgdir = /var/cache/yum/x86_64/2/epel/packages
priority = 99
proxy = False
proxy_dict =
proxy_password =
proxy_username =
repo_gpgcheck = False
report_instanceid = False
retries = 7
skip_if_unavailable = False
ssl_check_cert_permissions = True
sslcacert =
sslclientcert =
sslclientkey =
sslverify = True
throttle = 0
timeout = 5.0
ui_id = epel/x86_64
ui_repoid_vars = releasever,
   basearch
username =

- Erlang 설치

In [ ]:
[ec2-user@ip-10-1-10-188 ~]$ sudo yum install erlang --enablerepo=epel -y

- socat 설치

In [ ]:
[ec2-user@ip-10-1-10-188 ~]$ sudo yum install -y socat
Loaded plugins: extras_suggestions, langpacks, priorities, update-motd
194 packages excluded due to repository priority protections
Resolving Dependencies
--> Running transaction check
---> Package socat.x86_64 0:1.7.3.2-2.amzn2.0.1 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

====================================================================================================================
 Package               Arch                   Version                              Repository                  Size
====================================================================================================================
Installing:
 socat                 x86_64                 1.7.3.2-2.amzn2.0.1                  amzn2-core                 291 k

Transaction Summary
====================================================================================================================
Install  1 Package

Total download size: 291 k
Installed size: 1.1 M
Downloading packages:
socat-1.7.3.2-2.amzn2.0.1.x86_64.rpm                                                         | 291 kB  00:00:00
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : socat-1.7.3.2-2.amzn2.0.1.x86_64                                                                 1/1
  Verifying  : socat-1.7.3.2-2.amzn2.0.1.x86_64                                                                 1/1

Installed:
  socat.x86_64 0:1.7.3.2-2.amzn2.0.1

Complete!

- logrotate 설치

In [ ]:
[ec2-user@ip-10-1-10-188 ~]$ sudo yum install logrotate -y
Loaded plugins: extras_suggestions, langpacks, priorities, update-motd
194 packages excluded due to repository priority protections
Package logrotate-3.8.6-15.amzn2.x86_64 already installed and latest version
Nothing to do

- RabbitMq 설치 및 구동

http://www.rabbitmq.com/releases/rabbitmq-server 에서 안정화된 최신버전을 가져올건데 3.6.15 버전으로 설치할 것이다.

In [ ]:
[ec2-user@ip-10-1-10-56 ~]$ wget https://www.rabbitmq.com/releases/rabbitmq-server/v3.6.15/rabbitmq-server-3.6.15-1.el6.noarch.rpm
--2020-10-09 06:45:03--  https://www.rabbitmq.com/releases/rabbitmq-server/v3.6.15/rabbitmq-server-3.6.15-1.el6.noarch.rpm
Resolving www.rabbitmq.com (www.rabbitmq.com)... 104.20.11.224, 172.67.16.25, 104.20.10.224, ...
Connecting to www.rabbitmq.com (www.rabbitmq.com)|104.20.11.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5075605 (4.8M) [application/x-redhat-package-manager]
Saving to: ‘rabbitmq-server-3.6.15-1.el6.noarch.rpm’

100%[==================================================================================>] 5,075,605   6.69MB/s   in 0.7s

2020-10-09 06:45:05 (6.69 MB/s) - ‘rabbitmq-server-3.6.15-1.el6.noarch.rpm’ saved [5075605/5075605]

[ec2-user@ip-10-1-10-56 ~]$ sudo rpm -Uvh rabbitmq-server-3.6.15-1.el6.noarch.rpm
warning: rabbitmq-server-3.6.15-1.el6.noarch.rpm: Header V4 RSA/SHA1 Signature, key ID 6026dfca: NOKEY
Preparing...                          ################################# [100%]
Updating / installing...
   1:rabbitmq-server-3.6.15-1.el6     ################################# [100%]

그런 다음에 아래와 같이 RabbitMq 서버와 management 플러그인을 구동한다.

** 참고사항 : Management Plugin 란

"The RabbitMQ management plugin provides an HTTP-based API for management and monitoring of RabbitMQ nodes and clusters, along with a browser-based UI and a command line tool, rabbitmqadmin."

문구 출처 : https://www.rabbitmq.com/management.html

In [ ]:
[ec2-user@ip-10-1-10-40 ~]$ sudo systemctl start rabbitmq-server

[ec2-user@ip-10-1-10-40 ~]$ sudo rabbitmq-plugins enable rabbitmq_management
The following plugins have been enabled:
  amqp_client
  cowlib
  cowboy
  rabbitmq_web_dispatch
  rabbitmq_management_agent
  rabbitmq_management

Applying plugin configuration to rabbit@ip-10-1-10-40... started 6 plugins.

[ec2-user@ip-10-1-10-40 ~]$ sudo systemctl restart rabbitmq-server

[ec2-user@ip-10-1-10-40 ~]$ sudo systemctl status rabbitmq-server
● rabbitmq-server.service - LSB: Enable AMQP service provided by RabbitMQ broker
   Loaded: loaded (/etc/rc.d/init.d/rabbitmq-server; bad; vendor preset: disabled)
   Active: active (running) since Fri 2020-10-09 07:04:32 UTC; 6s ago
     Docs: man:systemd-sysv-generator(8)
  Process: 12653 ExecStop=/etc/rc.d/init.d/rabbitmq-server stop (code=exited, status=0/SUCCESS)
  Process: 13030 ExecStart=/etc/rc.d/init.d/rabbitmq-server start (code=exited, status=0/SUCCESS)
   CGroup: /system.slice/rabbitmq-server.service
           ├─13254 /bin/sh /etc/rc.d/init.d/rabbitmq-server start
           ├─13261 /bin/bash -c ulimit -S -c 0 >/dev/null 2>&1 ; /usr/sbin/rabbitmq-server
           └─13265 /bin/sh /usr/sbin/rabbitmq-server

Oct 09 07:04:27 ip-10-1-10-40.ap-northeast-2.compute.internal systemd[1]: Starting LSB: Enable AMQP service provided by......
Oct 09 07:04:27 ip-10-1-10-40.ap-northeast-2.compute.internal su[13122]: (to rabbitmq) root on none
Oct 09 07:04:28 ip-10-1-10-40.ap-northeast-2.compute.internal su[13279]: (to rabbitmq) root on none
Oct 09 07:04:28 ip-10-1-10-40.ap-northeast-2.compute.internal su[13281]: (to rabbitmq) root on none
Oct 09 07:04:32 ip-10-1-10-40.ap-northeast-2.compute.internal rabbitmq-server[13030]: Starting rabbitmq-server: SUCCESS
Oct 09 07:04:32 ip-10-1-10-40.ap-northeast-2.compute.internal rabbitmq-server[13030]: rabbitmq-server.
Oct 09 07:04:32 ip-10-1-10-40.ap-northeast-2.compute.internal systemd[1]: Started LSB: Enable AMQP service provided by ...er.
Hint: Some lines were ellipsized, use -l to show in full.

- RabbitMQ에 관리자계정 만들기

최초 설치시 guest 계정을 제공하기는 하지만 localhost에서만 접속을 허용하는 제한사항이 있다. 따라서 Management Plugin UI로 접근해서 설정하려면 관리자 계정을 사전에 만들어줘야 한다.

아래 예시는 관리자 계정을 rabbit_admin 이라는 이름으로 만들었다.

In [ ]:
[ec2-user@ip-10-1-10-40 ~]$ sudo rabbitmqctl add_user rabbit_admin MyNewStrongP@ssw0d!
Creating user "rabbit_admin"

[ec2-user@ip-10-1-10-40 ~]$ sudo rabbitmqctl set_user_tags rabbit_admin administrator
Setting tags for user "rabbit_admin" to [administrator]

계정이 잘 만들었는지 확인하기 위해 웹브라우저를 열고 `[EC 퍼블릭 아이피]:15672` 에 접속한다.

그리고 위에서 만든 계정으로 로그인하여 정상접속되는지 확인한다.